In [21]:
import pandas as pd
import numpy as np
import pickle
import re
import os
from os.path import join
from tqdm import tqdm
from datetime import datetime
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl import Workbook
from file_function import read_filelist, re_name_upload, write_log_csv_upload
wd = r'D:\scan'

### 시효완성일 > 시효완성월 작업하기

In [22]:
merged_raw = pd.read_excel(r"D:\3.자산\전산 dataset\솔림\230926\output1\POOL8_데이터_230926_발송.xlsx", sheet_name="무담보", dtype=str)

In [23]:
# 마지막 작업 : 시효완성월 전체 수정(새시효완성일에 대해)
basedate = "230926"
기준일 = pd.Timestamp("20"+basedate[:4]+"01")

# 함수벡터화
merged_raw["시효완성일"] = pd.to_datetime(merged_raw["시효완성일"])  # 시효완성일을 날짜 형식으로 변환
merged_raw["시효완성월_계산"] = (merged_raw["시효완성일"].dt.year - 기준일.year) * 12 + (merged_raw["시효완성일"].dt.month - 기준일.month)

# 버릴 시효완성월_ 작성
x = merged_raw["시효완성월_계산"]
conds = [(x<0) | pd.isna(x), x<1, x<2, x<3, x<4, x<5, x<6, x<12, x>=12]
values = ["시효만료", "당월도래", "1개월", "2개월", "3개월", "4개월", "5개월", "1년이내", "1년이후"]
merged_raw["시효완성월_계산"] = np.select(conds, values)

# 시효완성월과 시효사유 : 압류로 인한 중단 반영
merged_raw['시효완성월'] = np.where(merged_raw.시효완성월=="중단", merged_raw.시효완성월, merged_raw.시효완성월_계산)
merged_raw['시효사유'] = np.where(merged_raw.시효완성월=="중단", merged_raw.시효사유, merged_raw.시효사유)

# raw_data 저장 : 타입원복 및 불필요한 열 제거
# raw_data = merged_raw.iloc[:,:-5] # 시효확인필요, 시효완성일_, 시효사유_, 시효완성월_, 시효완성월_계산 제거
merged_raw["시효완성일"] = merged_raw["시효완성일"].apply(lambda x: pd.Timestamp(x).strftime("%Y-%m-%d") if pd.notna(x) else "")

merged_raw.to_excel(r"D:\3.자산\전산 dataset\솔림\230926\output1\xx.xlsx", index=False)

### 이름 순서대로 바꾸기 WITH 엑셀파일

In [ ]:
# 엑셀파일 읽기
##################################################
df_valid = pd.read_excel(join(r"D:\3.자산\프로젝트\2023 실물증서정리\광주\230828 스캔추가작업", "광주 스캔작업_230828.xlsx"), dtype=str).dropna(subset=['원인서류', '양도통지서', '법원문서'], how='all')
df_valid

In [ ]:
# 파일명 리스트 만들기
filename_list = []
for i, v in df_valid.iterrows() :
    filename = v.채무자키 + " " + v.채무자명
    if pd.notna(v.원인서류) :
        max = int(v.원인서류)
        for num in range(1, max+1) : 
            last_filename = filename + " 원인서류 " + str(num)
            filename_list.append(last_filename)
    if pd.notna(v.양도통지서) : 
        max = int(v.양도통지서)
        for num in range(1, max+1) : 
            last_filename = filename + " 양도통지서" + str(num)
            filename_list.append(last_filename)
    if pd.notna(v.법원문서) : 
        max = int(v.법원문서)
        for num in range(1, max+1) : 
            last_filename = filename + " 법원문서" + str(num)
            filename_list.append(last_filename)


In [ ]:
# 스캔파일 목록
path = wd
file_list = read_filelist(path) # 탐색기 정렬과 무관하게 이름 오름차순으로 정렬된 리스트 반환

In [ ]:
# 통합
df = pd.DataFrame({"after" : filename_list, "before" : file_list})

In [ ]:
# df 한행씩 불러와서 파일명 바꾸기
success, fail = [], []
for i, v in df.iterrows() :
    
    ext = os.path.splitext(v.before)[1]
    try :
        os.rename(join(path, v.before), join(path, v.after + ext))
        success.append([path, v.before, v.after])
    except Exception as e :
        fail.append([path, v.before, v.after, e.__class__])
        continue
print(f"총 {len(df)}개 중, {len(success)}개 파일명 변경, {len(fail)}개 파일명 변경 실패")

In [ ]:
df17[~df17.채무상태.str.contains("신용회복")&~df17.채무상태.str.contains("종결") & df17.담당자.str.contains("신용회복")][["채무자키", "채권구분","성명","채무상태","담당자","보증인수"]]

In [ ]:
path = r"\\192.168.0.75\솔림헬프\1.원인서류\전북-01"
p_extension = re.compile('녹취|m4a|wav|mp[34]') 
result = []
for root, dirs, files in os.walk(path) :
    for d in dirs : 
        if len(read_filelist(join(root, d))) == 0 :
            print(d)

In [ ]:
rst_df = pd.DataFrame(data=result, columns=["root", "filename"])
rst_df.to_excel('./녹취파일.xlsx', index=False)

In [ ]:
from os.path import join
import pickle
import pandas as pd

with open(join(path_dict, "outList.pkl"), 'rb') as pkl : outList = pickle.load(pkl)

In [ ]:
path_dict = r'\\192.168.0.75\스캔파일\스캔파일log\_project\대성파일\중복조사' 
with open(join(path_dict, "kcs별 파일정보_대성.pkl"), 'rb') as pkl : outList = pickle.load(pkl)
outList